In [11]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
import json
import itertools
from tqdm import tqdm

In [ ]:
def rossler_derivatives(state, t, a=0.2, b=0.2, c=5.7):
    """Compute time derivatives [dx/dt, dy/dt, dz/dt] for the Rössler system."""
    x, y, z = state
    dxdt = -y - z
    dydt = x + a * y
    dzdt = b + z * (x - c)
    return [dxdt, dydt, dzdt]

def generate_rossler_data(
    initial_state=[1.0, 0.0, 0.0],
    tmax=25.0,
    dt=0.01,
    a=0.2,
    b=0.2,
    c=5.7
):
    """
    Numerically integrate Rössler equations x'(t), y'(t), z'(t) using odeint.
    Returns:
       t_vals: array of time points
       sol   : array shape [num_steps, 3] of [x(t), y(t), z(t)]
    """
    num_steps = int(tmax / dt)
    t_vals = np.linspace(0, tmax, num_steps)
    sol = odeint(rossler_derivatives, initial_state, t_vals, args=(a, b, c))
    return t_vals, sol

In [14]:
def evaluate_nrmse(all_preds, test_target, horizons):
    """
    Evaluate model performance over multiple prediction horizons
    for teacher-forced single-step forecasting or autoregressive rollout.
    """
    horizon_nrmse = {}
    for horizon in horizons:
        preds = all_preds[:horizon]
        targets = test_target[:horizon]
        squared_errors = (preds - targets) ** 2
        variance = np.var(targets, axis=0)
        variance[variance == 0] = 1e-8  # avoid divide-by-zero
        nrmse = np.sqrt(np.sum(squared_errors) / (horizon * np.sum(variance)))
        horizon_nrmse[horizon] = nrmse
    return horizon_nrmse

In [15]:
def compute_valid_prediction_time(y_true, y_pred, t_vals, threshold, lambda_max, dt):
    """
    Compute the Valid Prediction Time (VPT) and compare it to Lyapunov time T_lambda = 1 / lambda_max.
    
    Parameters
    ----------
    y_true : ndarray of shape (N, dim)
        True trajectory over time.
    y_pred : ndarray of shape (N, dim)
        Model's predicted trajectory over time (closed-loop).
    t_vals : ndarray of shape (N,)
        Time values corresponding to the trajectory steps.
    threshold : float, optional
        The error threshold, default is 0.4 as in your snippet.
    lambda_max : float, optional
        Largest Lyapunov exponent. Default=0.9 for Lorenz.
        
    Returns
    -------
    T_VPT : float
        Valid prediction time. The earliest time at which normalized error surpasses threshold
        (or the last time if never surpassed).
    T_lambda : float
        Lyapunov time = 1 / lambda_max
    ratio : float
        How many Lyapunov times the model prediction remains valid, i.e. T_VPT / T_lambda.
    """
    # 1) Average of y_true
    y_mean = np.mean(y_true, axis=0)  # shape (dim,)
    
    # 2) Time-averaged norm^2 of (y_true - y_mean)
    y_centered = y_true - y_mean
    denom = np.mean(np.sum(y_centered**2, axis=1))  # scalar
    
    # 3) Compute the normalized error delta_gamma(t) = ||y_true - y_pred||^2 / denom
    diff = y_true - y_pred
    err_sq = np.sum(diff**2, axis=1)  # shape (N,)
    delta_gamma = err_sq / denom      # shape (N,)
    
    # 4) Find the first time index where delta_gamma(t) exceeds threshold
    idx_exceed = np.where(delta_gamma > threshold)[0]
    if len(idx_exceed) == 0:
        # never exceeds threshold => set T_VPT to the final time
        T_VPT = t_vals[-1]
    else:
        T_VPT = t_vals[idx_exceed[0]]
    
    # 5) Compute T_lambda and ratio
    T_lambda = 1.0 / lambda_max

    # print(f"\n--- Valid Prediction Time (VPT) with threshold={threshold}, lambda_max={lambda_max} ---")

    T_VPT = (T_VPT - t_vals[0])  # Adjust T_VPT to be relative to the start time
    ratio = T_VPT / T_lambda

    return T_VPT, T_lambda, ratio

In [ ]:
# def compute_attractor_deviation(predictions, targets, cube_size=(0.1, 0.1, 0.1)):
#     """
#     Compute the Attractor Deviation (ADev) metric.

#     Parameters:
#         predictions (numpy.ndarray): Predicted trajectories of shape (n, 3).
#         targets (numpy.ndarray): True trajectories of shape (n, 3).
#         cube_size (tuple): Dimensions of the cube (dx, dy, dz).

#     Returns:
#         float: The ADev metric.
#     """
#     # Define the cube grid based on the range of the data and cube size
#     min_coords = np.min(np.vstack((predictions, targets)), axis=0)
#     max_coords = np.max(np.vstack((predictions, targets)), axis=0)

#     # Create a grid of cubes
#     grid_shape = ((max_coords - min_coords) / cube_size).astype(int) + 1

#     # Initialize the cube occupancy arrays
#     pred_cubes = np.zeros(grid_shape, dtype=int)
#     target_cubes = np.zeros(grid_shape, dtype=int)

#     # Map trajectories to cubes
#     pred_indices = ((predictions - min_coords) / cube_size).astype(int)
#     target_indices = ((targets - min_coords) / cube_size).astype(int)

#     # Mark cubes visited by predictions and targets
#     for idx in pred_indices:
#         pred_cubes[tuple(idx)] = 1
#     for idx in target_indices:
#         target_cubes[tuple(idx)] = 1

#     # Compute the ADev metric
#     adev = np.sum(np.abs(pred_cubes - target_cubes))
#     return adev

In [17]:
def sigmoid(z: np.ndarray, /) -> np.ndarray:
    """Numerically-stable logistic σ : ℝ → (0, 1)."""
    out = np.empty_like(z, dtype=np.float32)
    np.subtract(0.0, z, out)            # out = −z   (no new allocation)
    np.exp(out, out)                    # out = e^(−z)
    out += 1.0                          # 1 + e^(−z)
    np.reciprocal(out, out)             # 1 / (1 + e^(−z))
    return out


def _sample_frequencies(m: int, mode: str = "log") -> np.ndarray:
    """
    Produce m distinct rotation angles θ_i ∈ (0, π).

    * 'log'  : log-uniform in [10⁰·⁰, 10⁰·⁹] Hz then mapped to θ = 2πfΔt
    * 'lin'  : uniform in (0, π)
    * 'gold' : golden-ratio spacing (deterministic)
    """
    if mode == "log":
        # log-spread over ~1 decade, then map to (0, π)
        f = 10 ** np.linspace(0.0, 0.9, m, dtype=np.float32)
        f /= f.max()
        return np.pi * f
    if mode == "lin":
        return np.random.default_rng().uniform(0.01, np.pi - 0.01, size=m)
    if mode == "gold":
        return (np.mod(np.arange(1, m + 1) * 0.61803398875, 1.0) * (np.pi - 0.02) + 0.01)
    raise ValueError("mode must be {'log','lin','gold'}")

def augment_state_with_squares(x):
    """
    Given state vector x in R^N, return [ x, x^2, 1 ] in R^(2N+1).
    We'll use this for both training and prediction.
    """
    x_sq = x**2
    return np.concatenate([x, x_sq, [1.0]])  # shape: 2N+1


# ---------------------------------------------------------------------
# Resonator reservoir class
# ---------------------------------------------------------------------
class FSR3D:
    """
    Frequency-Selective Resonator Echo-State Network.

    • N even ⇒ m = N/2 damped 2-D rotations (planar oscillators)
    • Exact spectral control: eigenvalues r·e^{±iθ_i}
    • Sparse symmetric nearest-neighbour coupling ε·C
    • Optional static gain profile g_k  (sin→σ)  for heterogeneity
    • Optional quadratic + quadrature feature map in the read-out
    """

    # -----------------------------------------------------------------
    def __init__(
        self,
        reservoir_size: int = 300,      # N (must be even)
        frequency_mode: str = "gold",    # how to draw θ_i
        r_damp: float = 0.95,           # attenuation per step
        eps_couple: float = 0.05,       # ε   cross-pair mixing strength
        input_scale: float = 0.5,
        leak_rate: float = 1.0,         # α
        ridge_alpha: float = 1e-6,
        use_gain: bool = True,
        gain_beta: float = 2.0,
        gain_sigmoid: bool = True,
        use_quadratic_feat: bool = True,
        seed: int = 42,
    ):
        if reservoir_size % 2:
            raise ValueError("reservoir_size must be even")
        self.N = reservoir_size
        self.m = reservoir_size // 2
        self.r = float(r_damp)
        self.eps = float(eps_couple)
        self.input_scale = input_scale
        self.alpha = leak_rate
        self.ridge_alpha = ridge_alpha
        self.use_gain = use_gain
        self.beta = gain_beta
        self.sig_gain = gain_sigmoid
        self.use_quad = use_quadratic_feat
        self.seed = seed
        self.freq_mode = frequency_mode

        # matrices and state
        self.W_res: np.ndarray | None = None
        self.W_in: np.ndarray | None = None
        self.W_out: np.ndarray | None = None
        self.g: np.ndarray | None = None
        self.x = np.zeros(self.N, dtype=np.float32)

        # one-off construction
        self._build_reservoir()
        self._build_gain()

    # -----------------------------------------------------------------
    # internal builders
    # -----------------------------------------------------------------
    def _build_reservoir(self):
        """
        Build   W_res = G · (R ⊕ ... ⊕ R  +  εC)   without the gain
        (gain applied separately for cheaper runtime multiplication).
        """
        m, r, eps = self.m, self.r, self.eps
        θ = _sample_frequencies(m, mode=self.freq_mode)

        # ----- block-diagonal damped rotations -----------------------
        R_blocks = np.zeros((self.N, self.N), dtype=np.float32)
        for i, theta in enumerate(θ):
            c, s = np.cos(theta) * r, np.sin(theta) * r
            i0 = 2 * i
            R_blocks[i0, i0] = c
            R_blocks[i0, i0 + 1] = -s
            R_blocks[i0 + 1, i0] = s
            R_blocks[i0 + 1, i0 + 1] = c

        # ----- sparse symmetric coupling -----------------------------
        rng = np.random.default_rng(self.seed)
        C = np.zeros_like(R_blocks)
        for i in range(m - 1):                        # nearest-neighbour chain
            b = rng.standard_normal()
            B = np.array([[b, 0.0], [0.0, b]], dtype=np.float32)
            a0, a1 = 2 * i, 2 * (i + 1)
            # upper block
            C[a0 : a0 + 2, a1 : a1 + 2] = B
            # symmetric lower block
            C[a1 : a1 + 2, a0 : a0 + 2] = B.T

        self.W_res = R_blocks + eps * C

    def _build_gain(self):
        """Static per-neuron gain g_k."""
        if not self.use_gain:
            self.g = np.ones(self.N, dtype=np.float32)
            return
        k_idx = np.arange(self.N, dtype=np.float32)
        raw = self.beta * np.sin(2.0 * np.pi * k_idx / self.N)
        self.g = sigmoid(raw) if self.sig_gain else raw.astype(np.float32)

    # -----------------------------------------------------------------
    # reservoir core
    # -----------------------------------------------------------------
    def _update(self, u_t: np.ndarray):
        pre = self.W_res @ self.x + self.W_in @ u_t
        if self.use_gain:
            pre *= self.g
        new_x = np.tanh(pre)
        self.x = (1.0 - self.alpha) * self.x + self.alpha * new_x

    def reset_state(self):
        self.x.fill(0.0)

    # -----------------------------------------------------------------
    # read-out training
    # -----------------------------------------------------------------
    def fit_readout(self, inputs: np.ndarray, targets: np.ndarray, discard: int = 100):
        """
        Teacher-forcing pass to learn W_out via ridge regression.
        * inputs  shape [T, d_in]
        * targets shape [T, d_out]
        """
        T, d_in = inputs.shape
        if T <= discard + 1:
            raise ValueError("Not enough data")

        # random input weights
        rng = np.random.default_rng(self.seed)
        self.W_in = (
            rng.uniform(-1.0, 1.0, size=(self.N, d_in)) * self.input_scale
        ).astype(np.float32)

        # collect echoed states
        self.reset_state()
        states = []
        for t in range(T):
            self._update(inputs[t])
            if t >= discard:
                states.append(self.x.copy())

        X = np.asarray(states, dtype=np.float32)            # [T−d, N]
        Y = targets[discard:]                               # align

        # feature map
        if self.use_quad:
            X_list = []
            for s in X:
                X_list.append(augment_state_with_squares(s))
            feats = np.array(X_list, dtype=np.float32)

        else:
            feats = X

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(feats, Y)
        self.W_out = reg.coef_.astype(np.float32)

    # -----------------------------------------------------------------
    # autoregressive forecast
    # -----------------------------------------------------------------
    def predict_autoregressive(self, init_u: np.ndarray, n_steps: int) -> np.ndarray:
        if self.W_out is None:
            raise RuntimeError("Call fit_readout() before prediction")

        d_in = init_u.shape[0]
        preds = np.empty((n_steps, self.W_out.shape[0]), dtype=np.float32)

        #self.reset_state()
        u_t = init_u.astype(np.float32).copy()

        for t in range(n_steps):
            self._update(u_t)

            if self.use_quad:
                feat_vec = augment_state_with_squares(self.x)
            else:
                feat_vec = self.x

            y_t = (self.W_out @ feat_vec).astype(np.float32)
            preds[t] = y_t
            u_t = y_t[:d_in]

        return preds
    
    def predict_open_loop(self, inputs: np.ndarray):
        preds = np.empty((inputs.shape[0], self.W_out.shape[0]), dtype=np.float32)
        for true_input in inputs:
            self._update(true_input)
            if self.use_quad:
                feat_vec = augment_state_with_squares(self.x)
            else:
                feat_vec = self.x
            out = (self.W_out @ feat_vec).astype(np.float32)
            preds.append(out)
        return preds


In [41]:
# grid = {
#     "input_scale": [0.7],
#     "ridge_alpha": [1e-8],
#     "r_damp": [0.5],
#     "eps_couple": [0.1],
#     "leak_rate": [0.7],
#     "gain_beta": [0.6]
# }

In [ ]:
grid = {
    "r_damp": [0.7],
    "eps_couple": [0.02],
    "input_scale": [0.2, 0.4, 0.6, 0.8, 1.0],
    "leak_rate": [0.1, 0.3, 0.5, 0.7, 0.9],
    "ridge_alpha": [1e-8],
    "gain_beta": [0.1, 0.3, 0.5, 0.7, 0.9],
}

In [ ]:
def run_grid_search(model_class, param_grid, model_name,
                    output_path="grid_search_results.json", f=generate_rossler_data, lambda_max=0.071):
    combos = list(itertools.product(*param_grid.values()))
    param_keys = list(param_grid.keys())
    print(f"\n== Initial grid search for {model_name} with {len(combos)} combinations ==")

    results = []
    # horizons = list(range(10, 1001, 10))
    horizons = [200, 400, 600, 800, 1000]
    

    for comb in tqdm(combos, desc="Grid Search"):
        params = dict(zip(param_keys, comb))
        seed_scores_vpt = []
        horizon_nrmse_all = {h: [] for h in horizons}
        horizon_nrmse_all_open_loop = {h: [] for h in horizons}
        adev_scores = []
        # ldev_scores = []

        for initial_state in [[1.0, 1.0, 1.0], [1.0, 2.0, 3.0], [2.0, 1.5, 4.0]]:
            tmax = 250
            dt = 0.02
            t_vals, lorenz_traj = f(
                initial_state=initial_state,
                tmax=tmax,
                dt=dt
            )

            washout = 2000
            t_vals = t_vals[washout:]
            lorenz_traj = lorenz_traj[washout:]

            scaler = MinMaxScaler()
            scaler.fit(lorenz_traj)
            lorenz_traj = scaler.transform(lorenz_traj)

            T_data = len(lorenz_traj)
            for train_frac in [0.3, 0.35, 0.4]:
                train_end = int(train_frac * (T_data - 1))
                train_input = lorenz_traj[:train_end]
                train_target = lorenz_traj[1:train_end + 1]
                test_input = lorenz_traj[train_end:-1]
                test_target = lorenz_traj[train_end + 1:]
                n_test_steps = len(test_input)
                initial_in = test_input[0]

                for seed in np.arange(1, 6):
                    model = model_class(**params, seed=seed)
                    model.fit_readout(train_input, train_target, discard=100)
                    preds = model.predict_autoregressive(initial_in, n_test_steps)
                    preds_open_loop = model.predict_open_loop(test_input)

                    T_VPT_s, _, ratio = compute_valid_prediction_time(test_target, preds, t_vals, 0.4, lambda_max, dt)
                    seed_scores_vpt.append(ratio)

                    horizon_nrmse = evaluate_nrmse(preds, test_target, horizons)
                    for h in horizons:
                        horizon_nrmse_all[h].append(horizon_nrmse[h])

                    horizon_nrmse_open_loop = evaluate_nrmse(preds_open_loop, test_target, horizons)
                    for h in horizons:
                        horizon_nrmse_all_open_loop[h].append(horizon_nrmse_open_loop[h])

                    # adev = compute_attractor_deviation(preds, test_target)
                    # adev_scores.append(adev)

                    # ldev = compute_lyapunov_exponent("Lorenz", preds, dt)
                    # ldev_scores.append(ldev)

        mean_vpt = float(np.mean(seed_scores_vpt))
        std_vpt = float(np.std(seed_scores_vpt))
        mean_nrmse_dict = {str(h): float(np.mean(horizon_nrmse_all[h])) for h in horizons}
        std_nrmse_dict  = {str(h): float(np.std(horizon_nrmse_all[h]))  for h in horizons}
        mean_nrmse_dict_open_loop = {str(h): float(np.mean(horizon_nrmse_all_open_loop[h])) for h in horizons}
        std_nrmse_dict_open_loop = {str(h): float(np.std(horizon_nrmse_all_open_loop[h])) for h in horizons}
        # mean_adev = float(np.mean(adev_scores))
        # std_adev = float(np.std(adev_scores))
        # mean_ldev = float(np.mean(ldev_scores))
        # std_ldev = float(np.std(ldev_scores))

        results.append({
            "params": params,
            "seed_scores_T_VPT": seed_scores_vpt,
            "mean_T_VPT": mean_vpt,
            "std_T_VPT": std_vpt,
            "mean_NRMSEs": mean_nrmse_dict,
            "std_NRMSEs": std_nrmse_dict,
            "mean_NRMSEs_open_loop": mean_nrmse_dict_open_loop,
            "std_NRMSEs_open_loop": std_nrmse_dict_open_loop,
            # Uncomment if you want to include ADev and LDev metrics
            # "mean_ADev": mean_adev,
            # "std_ADev": std_adev,
            # "mean_LDev": mean_ldev,
            # "std_LDev": std_ldev
        })

    with open(output_path, "w") as f:
        json.dump(results, f, indent=2)
    print(f"\nAll results saved to `{output_path}`")

    return results

In [ ]:
run_grid_search(FSR3D, grid, "fsr", output_path="fsr_r 41.json", f=generate_rossler_data, lambda_max=0.071)


== Initial grid search for fsr with 1 combinations ==


Grid Search: 100%|██████████| 1/1 [01:36<00:00, 96.30s/it]


All results saved to `fsr_l.json`


[{'params': {'input_scale': 0.7,
   'ridge_alpha': 1e-08,
   'r_damp': 0.5,
   'eps_couple': 0.1,
   'leak_rate': 0.7,
   'gain_beta': 0.6},
  'seed_scores_T_VPT': [11.268000000000002,
   4.752000000000001,
   4.770000000000004,
   4.770000000000004,
   4.680000000000002,
   11.411999999999999,
   11.502,
   10.133999999999999,
   10.026,
   10.170000000000003,
   9.540000000000001,
   8.244000000000003,
   8.892000000000001,
   8.244000000000003,
   9.504000000000001,
   10.404000000000002,
   8.333999999999998,
   8.297999999999998,
   8.297999999999998,
   10.494000000000003,
   11.159999999999998,
   7.344000000000003,
   10.962,
   11.033999999999997,
   7.182000000000003,
   8.388,
   8.316,
   8.442000000000002,
   8.442000000000002,
   10.962,
   7.164000000000001,
   7.182000000000003,
   10.259999999999998,
   7.182000000000003,
   11.682000000000004,
   7.002000000000001,
   6.966000000000001,
   6.911999999999999,
   10.782000000000004,
   10.709999999999999,
   8.171999999